In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("/Users/sanhorn/Desktop/Internships/Econ Data Lab/IFLS5 Dataset/b3a_tk3.csv")
df

,hhid14_9,pid14,tk28year,tk28,tk30x,tk31x,tk32x,tk33,tk33pid,tk33a,hhid14,pidlink,occ2014,tk31a,version,module
0,1060000,1,2007,1,3.0,3.0,3.0,2.0,NaN,3.0,10600,1060001,62,1.0,1.3,B3A_TK3
1,1060000,1,2008,1,3.0,3.0,3.0,2.0,NaN,3.0,10600,1060001,62,1.0,1.3,B3A_TK3
2,1060000,1,2009,1,1.0,1.0,1.0,2.0,NaN,3.0,10600,1060001,62,1.0,1.3,B3A_TK3
3,1060000,1,2010,3,NaN,NaN,NaN,NaN,NaN,NaN,10600,1060001,62,NaN,1.3,B3A_TK3
4,1060000,1,2011,3,NaN,NaN,NaN,NaN,NaN,NaN,10600,1060001,62,NaN,1.3,B3A_TK3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209747,321300000,3,2010,1,1.0,1.0,1.0,5.0,NaN,3.0,3213000,321300003,45,6.0,1.3,B3A_TK3
209748,321300000,3,2011,1,1.0,1.0,1.0,5.0,NaN,3.0,3213000,321300003,45,8.0,1.3,B3A_TK3
209749,321300000,3,2012,3,NaN,NaN,NaN,NaN,NaN,NaN,3213000,321300003,45,NaN,1.3,B3A_TK3
209750,321300000,3,2013,3,NaN,NaN,NaN,NaN,NaN,NaN,3213000,321300003,45,NaN,1.3,B3A_TK3


In [6]:
tmp_df = df.copy()
selected_columns = ["hhid14_9", "pid14", "tk28year", "tk28", "tk31a"]
tmp_df = tmp_df[selected_columns]

tmp_df["person"] = tmp_df["hhid14_9"].astype(str) + tmp_df["pid14"].astype(str)
tmp_df = tmp_df.drop(columns = ["hhid14_9", "pid14"])
tmp_df

,tk28year,tk28,tk31a,person
0,2007,1,1.0,10600001
1,2008,1,1.0,10600001
2,2009,1,1.0,10600001
3,2010,3,NaN,10600001
4,2011,3,NaN,10600001
...,...,...,...,...
209747,2010,1,6.0,3213000003
209748,2011,1,8.0,3213000003
209749,2012,3,NaN,3213000003
209750,2013,3,NaN,3213000003


In [9]:
tmp_df["tk31a"].isna().sum()

60570

In [5]:
print("tk31a NaN number: " + str(len(tmp_df[tmp_df["tk31a"].isna()])))
print("tk28 doesn't have job number: " + str(len(tmp_df[tmp_df["tk28"] != 1])))

tk31a NaN number: 60570
tk28 doesn't have job number: 60570


In [7]:
mapping = {
    1: "Agriculture, forestry, fishing and hunting",
    2: "Mining and quarrying",
    3: "Manufacturing",
    4: "Electricity, gas, water",
    5: "Construction",
    6: "Wholesale, retail, restaurants and hotels",
    7: "Transportation, storage and communications",
    8: "Finance, insurance, real estate and business services",
    9: "Social services",
    10: "Activities that cannot be classified"
}

In [8]:
df1 = tmp_df.copy()

years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
sector_dfs = {}
for i in range(1, 10):
    sector_dfs[i] = pd.DataFrame(columns=years)

no_job_df = pd.DataFrame(columns=years)

for _, row in df1.iterrows():
    year = str(row["tk28year"])
    person_id = str(row["person"])
    person_hasJob = row["tk28"]

    if person_hasJob == 1:
        sector = int(row["tk31a"])

        if sector in sector_dfs:
            if year in sector_dfs[sector].columns:
                if sector_dfs[sector].empty:
                    sector_dfs[sector].loc[0] = [[] for _ in range(len(years))]
                curr = sector_dfs[sector].at[0, year]
                if isinstance(curr, list): curr.append(person_id)
                else: curr = [person_id]

                sector_dfs[sector].at[0, year] = curr

    if person_hasJob != 1:
        if no_job_df.empty:
            no_job_df.loc[0] = [[] for _ in range(len(years))]
        
        curr = no_job_df.at[0, year]
        if isinstance(curr, list): curr.append(person_id)
        else: curr = [person_id]

        no_job_df.at[0, year] = curr


In [11]:
import pandas as pd

# Initialize the table with sector names and "No Job" for the y-axis
table_data = {year: [] for year in years}
sector_names = [mapping[i] for i in range(1, 10)] + ["No Job"]

# Populate the table with lengths of lists in sector_dfs
for i in range(1, 10):
    for year in years:
        if not sector_dfs[i].empty and year in sector_dfs[i].columns:
            table_data[year].append(len(sector_dfs[i].at[0, year]))
        else:
            table_data[year].append(0)

# Add lengths of lists in no_job_df
for year in years:
    if not no_job_df.empty and year in no_job_df.columns:
        table_data[year].append(len(no_job_df.at[0, year]))
    else:
        table_data[year].append(0)

# Create the DataFrame
result_table = pd.DataFrame(table_data, index=sector_names)
result_table

,2007,2008,2009,2010,2011,2012,2013,2014
"Agriculture, forestry, fishing and hunting",5519,5605,5738,5857,5972,6072,6103,2250
Mining and quarrying,189,192,199,227,236,242,245,105
Manufacturing,2184,2251,2301,2384,2482,2622,2764,1649
"Electricity, gas, water",63,79,86,87,97,97,107,67
Construction,863,895,893,941,971,1022,1087,556
"Wholesale, retail, restaurants and hotels",3778,3946,4154,4361,4634,4955,5349,3024
"Transportation, storage and communications",512,506,525,542,556,558,564,280
"Finance, insurance, real estate and business services",445,474,474,519,520,567,585,311
Social services,3809,3913,4131,4326,4462,4642,4904,2773
No Job,8527,8027,7358,6609,5905,5036,4083,15025
